Once the graph state reaches the `END` all the context to the model is lost and would not be able to answer the question connected to the previous conversation.

Hence there needs to be a way to store the graph at every point.

In this regard we can use MemorySaver checkpointer, a in-memory solution or we can use Postgres for longer persistence.



In [2]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model = "qwen2.5:7b")

In [4]:
def agent(state):
    return {"messages": llm.invoke(state["messages"])}

In [8]:
from langgraph.graph import StateGraph, MessagesState, START

builder = StateGraph(MessagesState)
builder.add_node("agent_node", agent)

builder.add_edge(START, "agent_node")
graph = builder.compile()

In [7]:
input_message = {"role": "user", "content": "hi! my name is qwerty"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

input_message = {"role": "user", "content": "what's my name?"}
for chunk in graph.stream({"messages": [input_message]}, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! my name is qwerty
================================== Ai Message ==================================

Hello qwerty! It's nice to meet you. How can I assist you today? Feel free to ask me any questions or let me know if there's anything specific you'd like to discuss.
================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name was not provided in the question. Could you please tell me your name so I can assist you better?


The graph did not remember my name across multiple invokes. We need to add some kind of persistence for it to remember.

In [9]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
graph = builder.compile(checkpointer= memory)

In [10]:
config = {"configurable": {"thread_id": "1"}}
input_message = {"role": "user", "content": "hi! my name is qwerty"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

hi! my name is qwerty
================================== Ai Message ==================================

Hello qwerty! Nice to meet you. How can I assist you today? Is there anything specific you'd like to talk about or any questions you have?


In [11]:
input_message = {"role": "user", "content": "what's my name?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

what's my name?
================================== Ai Message ==================================

Your name is qwerty. How about we use a more common name for our conversation, or do you prefer to keep it as qwerty?


We can pass in different thread id to start a new conversation.